In [1]:
def loadDataSet():
    return [[1,3,4],[2,3,5],[1,2,3,5],[2,5]]

In [2]:
def createC1(dataSet):
    C1 = []#存储所有不重复的项值
    for transaction in dataSet:#数据集中的交易记录
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    # 映射为frozenset唯一性的，可使用其构造字典
    return list(map(frozenset, C1)) #map(function, iterable, ...)返回一个迭代器
''' 
map(square, [1,2,3,4,5])   # 计算列表各个元素的平方
[1, 4, 9, 16, 25]
>>> map(lambda x: x ** 2, [1, 2, 3, 4, 5])  # 使用 lambda 匿名函数
[1, 4, 9, 16, 25]
 
# 提供了两个列表，对相同位置的列表数据进行相加
>>> map(lambda x, y: x + y, [1, 3, 5, 7, 9], [2, 4, 6, 8, 10])
[3, 7, 11, 15, 19]
'''
'''
set(可变集合)与frozenset(不可变集合)的区别：
set无序排序且不重复，是可变的，有add（），remove（）等方法。
既然是可变的，所以它不存在哈希值。基本功能包括关系测试和消除重复元素.
集合对象还支持union(联合), intersection(交集), difference(差集)和sysmmetric difference(对称差集)等数学运算. 
sets 支持 x in set, len(set),和 for x in set。作为一个无序的集合，sets不记录元素位置或者插入点。
因此，sets不支持 indexing, 或其它类序列的操作。
frozenset是冻结的集合，它是不可变的，存在哈希值，好处是它可以作为字典的key，也可以作为其它集合的元素。
缺点是一旦创建便不能更改，没有add，remove方法。
'''

In [3]:
def scanD(D, Ck, minSupport):#数据集，候选项集列表，最小支持度
    ssCnt = {}
    for tid in D:#遍历数据集中的所有交易记录
        for can in Ck:#遍历C1中的所有候选集
            if can.issubset(tid):#测试是否 can 中的每一个元素都在 tid 中 
                if not can in ssCnt:#确定给定的键是否存在于字典中，如果存在则返回true，否则返回false。
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            retList.insert(0, key)#在列表的首部插入新的集合
            supportData[key] = support  
    return retList, supportData

In [4]:
dataSet=loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [5]:
C1=createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [6]:
D=list(map(set,dataSet))
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [7]:
L1=scanD(D,C1,0.5)
L1

([frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 {frozenset({1}): 0.5,
  frozenset({3}): 0.75,
  frozenset({2}): 0.75,
  frozenset({5}): 0.75})

In [10]:
ssCnt = {}
for tid in D:#遍历数据集中的所有交易记录
    for can in C1:#遍历C1中的所有候选集
        if can.issubset(tid):#测试是否 can 中的每一个元素都在 tid 中 
            if not can in ssCnt:#确定给定的键是否存在于字典中，如果存在则返回true，否则返回false。
                ssCnt[can] = 1
            else:
                ssCnt[can] += 1

In [11]:
ssCnt

{frozenset({1}): 2,
 frozenset({3}): 3,
 frozenset({4}): 1,
 frozenset({2}): 3,
 frozenset({5}): 3}

In [ ]:
def aprioriGen(Lk, k):  # 创建候选K项集 ##LK为频繁K项集
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            L1 = list(Lk[i])[:k - 2]
            L2 = list(Lk[j])[:k - 2]
            L1.sort()
            L2.sort()
            if L1 == L2:  # 前k-1项相等，则可相乘，这样可防止重复项出现
                #  进行剪枝（a1为k项集中的一个元素，b为它的所有k-1项子集）
                a = Lk[i] | Lk[j]  # a为frozenset()集合
                a1 = list(a)
                b = []
                # 遍历取出每一个元素，转换为set，依次从a1中剔除该元素，并加入到b中
                for q in range(len(a1)):
                    t = [a1[q]]
                    tt = frozenset(set(a1) - set(t))
                    b.append(tt)
                t = 0
                for w in b:
                    # 当b（即所有k-1项子集）都是Lk（频繁的）的子集，则保留，否则删除。
                    if w in Lk:
                        t += 1
                if t == len(b):
                    retList.append(b[0] | b[1])
    return retList
 

In [ ]:
'''
使用 aprioriGen() 来创建 Ck ，
然后使用 scanD() 基于 Ck 来创建 Lk 。
Ck 是一个候选项集列表，
然后 scanD() 会遍历 Ck 
丢掉不满足最小支持度要求的项集。
Lk 列表被添加到 L ，同时增加 k 的值
重复上述过程。最后，当 Lk 为空时，程序返回L并退出。'''

In [12]:
def aprioriGen(Lk, k): #频繁项集列表LK，项集元素个数K
    retList = []#建立空列表
    lenLk = len(Lk)#计算lk中的元素数目
    for i in range(lenLk):#比较LK中的每一个元素与其他元素
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2];
            L2 = list(Lk[j])[:k-2]
            L1.sort(); 
            L2.sort()
            if L1==L2: #取列表中的两个集合进行比较if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #合并这两个集合
    return retList

In [15]:
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)#创建L1
    L = [L1]#将L1放入列表L中
    k = 2
    while (len(L[k-2]) > 0):#创建包含更大项集的更大列表，直到下一个大的项集为空
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [16]:
L,suppData=apriori(dataSet)
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [18]:
aprioriGen(L[0],2)

[frozenset({2, 5}),
 frozenset({3, 5}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({1, 2}),
 frozenset({1, 3})]